In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import nltk
from sklearn import *
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import os
#os.chdir('Gender_Pronoun')
os.chdir("C:\\Users\\pavan samanth\\Gender_Pronoun")

C:\Users\pavan samanth\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\pavan samanth\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\pavan samanth\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20

In [2]:
test=pd.read_csv('test_stage_1.tsv',delimiter='\t',encoding='utf-8').rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})

In [3]:
test.head()

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,B_Noun,B-offset,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [4]:
development=pd.read_csv('gap-development.tsv.txt',delimiter='\t',encoding='utf-8')

In [5]:
development.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [6]:
validation = pd.read_csv('gap-validation.tsv.txt',delimiter='\t')

In [7]:
validation.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,validation-1,He admitted making four trips to China and pla...,him,256,Jose de Venecia Jr,208,False,Abalos,241,False,http://en.wikipedia.org/wiki/Commission_on_Ele...
1,validation-2,"Kathleen Nott was born in Camberwell, London. ...",She,185,Ellen,110,False,Kathleen,150,True,http://en.wikipedia.org/wiki/Kathleen_Nott
2,validation-3,"When she returns to her hotel room, a Liberian...",his,435,Jason Scott Lee,383,False,Danny,406,True,http://en.wikipedia.org/wiki/Hawaii_Five-0_(20...
3,validation-4,"On 19 March 2007, during a campaign appearance...",he,333,Reucassel,300,True,Debnam,325,False,http://en.wikipedia.org/wiki/Craig_Reucassel
4,validation-5,"By this time, Karen Blixen had separated from ...",she,427,Finch Hatton,290,False,Beryl Markham,328,True,http://en.wikipedia.org/wiki/Denys_Finch_Hatton


In [8]:
#train = pd.concat((test,validation)).rename(columns={'A': 'A_Noun', 'B': 'B_Noun'}).reset_index(drop=True)

In [9]:
#train.shape

In [10]:
gh_test = pd.read_csv('gap-test.tsv.txt',delimiter='\t')

In [11]:
gh_test.shape

(2000, 11)

In [12]:
train = pd.concat((gh_test,validation)).rename(columns={'A': 'A_Noun', 'B': 'B_Noun'}).reset_index(drop=True)

In [13]:
train.shape


(2454, 11)

In [14]:
train.to_excel("train_gap.xlsx")

C:\Users\pavan samanth\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://www.orangeprize.co.uk/opf/archive.php4%20In%202000,%20Hamilton%20was%20named%20a%20Notable%20Wisconsin%20Author%20by%20the%20Wisconsin%20Library%20Association.%20http://www.wla.lib.wi.us/readers/WLAC/Notable/Notable2000.htm%20All%20of%20her%20books%20are%20set,%20at%20least%20in%20part,%20in%20Wisconsin.%20%60%60A%20Map%20of%20the%20World''%20is%20set%20in%20Racine%20County,%20Wis.%20Of%20her%20writing,%20novelist%20Laura%20Moriarty%20says:%20%60%60I%20like%20Jane%20Hamilton%20for%20her%20compassionate%20portrayals%20of%20characters%20most%20people%20would%20ridicule,%20and%20the%20way%20her%20books%20show%20the%20beauty%20of%20rural%20life%20without%20romanticizing%20it.''' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


In [15]:
def name_replace(s, r1, r2):
    s = str(s).replace(r1,r2)
    for r3 in r1.split(' '):
        s = str(s).replace(r3,r2)
    return s

In [16]:
train['Text'] = train.apply(lambda r: name_replace(r['Text'], r['A_Noun'], 'subjectone'), axis=1)
#df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['B_Noun'], 'subjecttwo'), axis=1)

In [17]:
train['Text']

0       Upon their acceptance into the Kontinental Hoc...
1       Between the years 1979-1981, River won four lo...
2       Though his emigration from the country has aff...
3       At the trial, Pisciotta said: ``Those who have...
4       It is about a pair of United States Navy shore...
5       The others were Adam Baldwin (Jayne Cobb in Fi...
6       subjectone Fischer (born October 19, 1988) is ...
7       The monster arrives and bites Jeni's tongue, b...
8       On June 4, 1973 at the Felt Forum, Madison Squ...
9       Go Away (Lorrie Morgan song) ``Go Away'' is a ...
10      Taken in by S.H.I.E.L.D. she is under the dire...
11      Neither knows the truth about their respective...
12      He was a lawyer in Pulaski County before servi...
13      He left the Army in 1946. As the recipient of ...
14      subjectone is a British actress best known for...
15      Indeed, Buster was fully intended to exist in ...
16      In the late 1970s, she toured in musical comed...
17      Jennif

In [18]:
def name_replace(s, r1, r2):
    s = str(s).replace(r1,r2)
    for r3 in r1.split(' '):
        s = str(s).replace(r3,r2)
    return s

def get_features(df):
    df['section_min'] = df[['Pronoun-offset', 'A-offset', 'B-offset']].min(axis=1)
    df['Pronoun-offset2'] = df['Pronoun-offset'] + df['Pronoun'].map(len)
    df['A-offset2'] = df['A-offset'] + df['A_Noun'].map(len)
    df['B-offset2'] = df['B-offset'] + df['B_Noun'].map(len)                               
    df['section_max'] = df[['Pronoun-offset2', 'A-offset2', 'B-offset2']].max(axis=1)
    #df['Text'] = df.apply(lambda r: r['Text'][: r['Pronoun-offset']] + 'pronountarget' + r['Text'][r['Pronoun-offset'] + len(str(r['Pronoun'])): ], axis=1)
    df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['A_Noun'], 'subjectone'), axis=1)
    df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['B_Noun'], 'subjecttwo'), axis=1)
    
    
    df['A-dist'] = (df['Pronoun-offset'] - df['A-offset']).abs()
    df['B-dist'] = (df['Pronoun-offset'] - df['B-offset']).abs()
    return(df)

train = get_features(train)
test = get_features(test)

In [19]:
train

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,A-coref,B_Noun,B-offset,B-coref,URL,section_min,Pronoun-offset2,A-offset2,B-offset2,section_max,A-dist,B-dist
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner,352,386,361,372,386,31,17
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso,353,433,359,408,433,77,40
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh,256,314,268,301,314,56,17
3,test-4,"At the trial, subjecttwo said: ``Those who hav...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,377,529,384,545,545,149,10
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers,406,409,426,570,570,15,153
5,test-6,The others were Adam Baldwin (Jayne Cobb in Fi...,her,349,Jewel Staite,281,True,Keller,310,False,http://en.wikipedia.org/wiki/Jennifer_Keller,281,352,293,316,352,68,39
6,test-7,"subjectone Fischer (born October 19, 1988) is ...",She,365,Allison,232,True,Grace Smythe,290,False,http://en.wikipedia.org/wiki/Allison_Fischer,232,368,239,302,368,133,75
7,test-8,The monster arrives and bites subjecttwo's ton...,her,307,Sophie,248,False,Jeni,277,True,http://en.wikipedia.org/wiki/Leprechaun:_Origins,248,310,254,281,310,59,30
8,test-9,"On June 4, 1973 at the Felt Forum, Madison Squ...",he,227,Malave,124,True,Greg Joiner,169,False,http://en.wikipedia.org/wiki/Edwin_Malave,124,229,130,180,229,103,58
9,test-10,Go Away (subjecttwo song) ``Go Away'' is a son...,her,223,Cathy Majeski,78,False,Lorrie Morgan,154,True,http://en.wikipedia.org/wiki/Go_Away_(Lorrie_M...,78,226,91,167,226,145,69


In [20]:
def get_nlp_features(s, w):
    doc = nlp(str(s))
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    return len(tokens[((tokens['text']==w) & (tokens['dep']=='poss'))])

train['A-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'subjectone'))
train['B-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'subjecttwo'))
test['A-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'subjectone'))
test['B-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'subjecttwo'))


In [21]:
train = train.rename(columns={'A-coref':'A', 'B-coref':'B'})
train['A'] = train['A'].astype(int)
train['B'] = train['B'].astype(int)
train['NEITHER'] = 1.0 - (train['A'] + train['B'])

In [22]:
col = ['Pronoun-offset', 'A-offset', 'B-offset', 'section_min', 'Pronoun-offset2', 'A-offset2', 'B-offset2', 'section_max', 'A-poss', 'B-poss', 'A-dist', 'B-dist']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)
x1.head()

,Pronoun-offset,A-offset,B-offset,section_min,Pronoun-offset2,A-offset2,B-offset2,section_max,A-poss,B-poss,A-dist,B-dist
1699,448,379,390,379,451,385,401,451,0,0,69,58
1790,374,315,346,315,377,329,358,377,0,2,59,28
1665,349,297,329,297,352,306,334,352,0,0,52,20
898,299,262,281,262,302,265,285,302,0,1,37,18
270,431,336,379,336,433,340,385,433,0,1,95,52


In [23]:
model = multiclass.OneVsRestClassifier(ensemble.RandomForestClassifier(max_depth = 7, n_estimators=1000, random_state=33))
# model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

# param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
# model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

log_loss 0.9241298387965972


In [24]:
test.shape

(2000, 21)

In [25]:
submission = pd.read_csv('submission.csv')

In [26]:
submission

,ID,A,B,NEITHER
0,development-1,0.405355,0.458921,0.104922
1,development-2,0.434218,0.476733,0.096633
2,development-3,0.475165,0.454440,0.073557
3,development-4,0.505004,0.423239,0.080587
4,development-5,0.446601,0.435365,0.104820
5,development-6,0.470788,0.437313,0.074523
6,development-7,0.469262,0.465283,0.073281
7,development-8,0.500614,0.415727,0.106587
8,development-9,0.393313,0.513080,0.119463
9,development-10,0.472862,0.415946,0.134474
